#1.load the data

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import json

from datetime import timedelta
from tqdm.notebook import tqdm
from collection import Counter
from heapq import nlargest

#그림 그리기
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_theme()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_path = Path("/kaggle/input/otto-recommender-system")

train= data_path/'train.jsonl'
test= data_path/'test.jsonl'

sample_sub = Path("/kaggle/input/otto-recommender-system/sample_submission.csv")

In [ ]:
with open(train,'r') as f :
  print(f'total lines in train data is : {len(f.readlines())}')
  

#2. sample_looking

2-1. events

In [ ]:
sample_size =5

chunks = pd.read_json(train, lines=True, chunksize=sample_size)

for chunk in chunks :
  sample_train_df = chunk
  break

In [ ]:
sample_train_df.set_index('session', drop=True, inplace=True)

for i, row in tqdm(sample_train_df.iterrows(), total=len(sample_train_df)) :
  actions = row['events']
  print(actions)

In [ ]:
#extract first session of sample_train_df

example_session = sample_train_df.iloc[0].item()

example_session[0]

In [ ]:
#time of session
time_elapsed = example_session[-1]['ts'] - example_session[0]['ts']


#하나의 세션에서 dict의 개수 추출
for action in example_session :
  action_counts[action['type']] = action_counts.get(action['type'],0) +1

In [ ]:
#extract info from each session and make it to the df

action_counts_list, article_id_list, session_length_ts_list, session_length_action_list = [[] for i in range(4)]

overall_action_counts = {}
overall_article_id_counts = {}

for i, row in tqdm(sample_df_train_df.iterrows(), total=len(sample_df_train_df)) :
  actions = row['events']

  action_counts = {}
  article_id_counts = {}

  action in actions :
  action_counts[action['type']] = action_counts.get(action['type'],0) +1
  article_id_counts[action['aid']] = article_id_counts.get(action['aid'],0) +1

  overall_action_counts[action['type']] = overall_action_counts.get(action['type'],0) +1
  overall_article_id_counts[action['aid']] = overall_article_id_counts.get(actio['aid'],0) +1

  session_length_time = action['ts'][-1] - action['ts'][0]

  #add to list
  action_counts_list.append(action_counts)
  article_id_list.append(article_id_counts)
  session_length_ts_list.append(session_length_time)
  session_length_action_list.append(len(actions))

sample_train_df['action_counts'] = action_counts_list
sample_train_df['article_id_list'] =article_id_list
sample_train_df['session_length_time'] =session_length_time_list
sample_train_df['session_length_hours'] = sample_train_df['session_length_time']*2.77778e-7
sample_train_df['session_length_action'] = session_length_action_list

In [ ]:
total_actions = sum(overall_action_counts.values())

plt.figrue(figsize=(8,6))
sns.barplot(overall_action_counts.keys(), [i/total_actions for i in overall_action_counts.values()])
plt.title('action')
plt.xlabel('type')
plt.ylabel('counts')
plt.show()

##2-1. check null values

In recommendation systems, we do nto prefer to use stuff like fillna.

instead, we make kind of most frequent(popularity of all).

##2-2.data optimization

####Baseline

2-2-1. type

In [ ]:
with open(test, 'r') as f :
  print(f'test data has {len(f.readlines())} lines')

In [ ]:
sample_size=1000

chunks = pd.read_json(test, lines=True, chunksize=sample_size)

for chunk in chunks :
  sample_test_df = chunk
  break

In [ ]:
sample_subsmission = pd.read_csv(sample_sub)

In [ ]:
sample_size =15000

chunks = pd.read_json(train, lines=True, chunksize=sample_size)

clicks_article_list = []
carts_article_list = []
orders_article_list = []

for session, events in enumerate(chunks) :
  if session >2 :
    break
  
  sample_train_df = events

  for i, row in events.iterrows() :
    actions = row['events']

    for action in actions :
      if action['type'] == 'clicks':
        clicks_article_list.append(action['aid'])
      elif action['type'] == 'carts':
        carts_article_list.append(action['aid'])
      else:
        orders_article_list.append(action['aid'])

In [ ]:
clicks_article_freq = Counter(clicks_article_list)
carts_article_freq = Counter(carts_article_list)
orders_article_freq = Counter(orders_article_list)

In [ ]:
top_clicks_article = nlargest(20, clicks_article_freq, key=clicks_article_freq.get)
top_carts_article = nlargest(20, carts_article_freq, key=carts_article_freq.get)
top_orders_article = nlargest(20, orders_article_freq, key=orders_article_freq.get)

In [ ]:
frequent_articles = {'clicks': top_click_article, 'carts':top_carts_article, 'order':top_order_article}

In [ ]:
for action in ['clicks','carts','order'] :
  print(f'most frequent articles for {action} : {frequent_articles[action]}')

In [ ]:
test_data = pd.read_json(test, lines=True, chunksize=1000)

preds = []

for chunk in tqdm(test, total=len(test)) :

  for i, row in chunk.iterrows() :
    actions = row['events']
    article_id_list = []

    for action in actions :
      article_id_list.append(action['aid'])

    article_freq = Counter(article_id_list)
    top_articles = nlargest(20, article_freq, key=article_freq)

    #pad with most popular items in training
    padding_size = -(20 - len(top_articles))
    for action in ['clicks','carts','order'] :
      top_articles = top_articles + frequent_articles[action][padding_size:]
      preds.append(" ".join([str(id) for id in top_articles]))

In [ ]:
sample_submission['labels'] = preds

sample_submission.to_csv('submission.csv', index=False)

###2-2-1. Word2Vec

reference : https://www.kaggle.com/code/takaito/otto-word2vec-tutorial

#3. Ranking Method

- with XGB : https://medium.com/predictly-on-tech/learning-to-rank-using-xgboost-83de0166229d

- with NN : https://towardsdatascience.com/learning-to-rank-with-python-scikit-learn-327a5cfd81f

1. 데이터를 불러오기
2. word2vec을 통한 vectorization
3. vectorization된 결과를 바탕으로 xgboosting(or NNModeling)
- 이때, weight=각각 0.1,0.3,0.6으로
- 예를들어, model = xgb.XGBRanker(  
    tree_method='gpu_hist',
    booster='gbtree',
    objective='rank:pairwise',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
    )

model.fit(X_train, y_train, group=groups, verbose=True)
4. 추가적으로 hyper parameter를 학습시키고싶으면 
grid search(bayesian optimization이 더 효과적)사용
- https://www.cognex.com/ko-kr/blogs/deep-learning/research/overview-bayesian-optimization-effective-hyperparameter-search-technique-deep-learning-2


##XGB

In [ ]:
from nvtabular import *
from merlin.schema.tags import Tags
import polars as pl
import xgboost as xgv

from merlin.core.utils import Distributed
from merlin.models.xgb import XGBoost
from nvtabular.ops import AddTags

## bayesian optimization

reference : https://www.cognex.com/ko-kr/blogs/deep-learning/research/overview-bayesian-optimization-effective-hyperparameter-search-technique-deep-learning-2

In [ ]:
from bayes_opt import BayesianOptimization
import numpy as np

1. 목적 함수 정의

In [ ]:
def target(x) :
  return np.exp(-(x-3)**2) + np.exp(-(3*x-2)**2) + 1/x**2+1

2. BayesianOptimization 객체 생성

Bayesianopt의 객체내에는 Surrogate Model인 GP가 내장되어있으며, _gp로 표현됨

- target : 목적함수 f(x)
- {'x' : (-2,6)} : 입력값 x의 탐색 대상 구간의 dict

In [ ]:
bayes_optimizer = BayesianOPtimization(target, {'x' : (-2,6)}, randon_state=42)

3. 실행

- init_points : 맨 처음 일부 Random Search방법으로 조사할 입력값의 갯수
- acq : Acquisition Function 선택 //
- acq('ei') : Expected Improvement
- xi : exploration-explotation간의 상대적 강도를 조절해주는 파라미터

In [ ]:
bayes_optimizer.maximize(init_points=2, n_iter=14, acq='ei', xi=0.01)

##statistics

- Support
- Confidence
- Lift

3-2. Apriori

3-3. Similarity

- cosine similarity
- pearson corr

#Collaboritive Filtering

- User Based
- Content based


##3-5. Word2Vec

reference : https://www.kaggle.com/code/radek1/word2vec-how-to-training-and-submission

##3-6. Filtering with model




NeuralNet

reference : https://towardsdatascience.com/learning-to-rank-with-python-scikit-learn-327a5cfd81f

#4. Modeling with hugging_face



##4-1. make my data to train/test split

https://www.kaggle.com/code/theoviel/pretraining-with-merlin-s-transformers4rec